# Calculate Alpha, Beta and CAPM

In [100]:
# import libraries

import os.path
import numpy as np
import pandas as pd
import pandas_datareader as web
import hvplot.pandas
from pathlib import Path

%matplotlib inline


In [56]:
# intitialize variables

index_stocks = ['AGG', 'SPY', "QQQ"]
picked_stocks = ['ACB', 'T', 'GOOG', 'BA']
random_stocks = ['GAIA', 'FMS', 'RC']

combined_stocks = index_stocks + picked_stocks + random_stocks
combined_stocks.sort()

start = pd.to_datetime('2015-01-01')
end = pd.to_datetime('2019-12-31')


In [64]:
# Create function to pull finance data, and save to csv, then 
stock_df = f"./csv/{combined_stocks[0]}-to-{combined_stocks[-1]}-from-{start.date()}-to-{end.date()}.csv"

if os.path.exists(stock_df): #if the csv file exists in directory, load file

    print(f"{stock_df} exists in this directory")

    stocks = pd.read_csv(stock_df)

    stocks.set_index("Date", inplace=True)

else: #otherwise pull fresh data

    print(f"Expected file does not exist, so a new file was created at {stock_df}.")

    stock_data = web.DataReader(combined_stocks, 'yahoo', start, end)

    stocks = stock_data["Close"]

    stocks.to_csv(f"./csv/{combined_stocks[0]}-to-{combined_stocks[-1]}-from-{start.date()}-to-{end.date()}.csv")


./csv/ACB-to-T-from-2015-01-01-to-2019-12-31.csv exists in this directory


In [68]:
stocks[random_stocks].head(10)

,GAIA,FMS,RC
Date,,,
2015-01-02,7.02,36.549999,17.510000
2015-01-05,7.06,36.459999,17.500000
2015-01-06,7.00,36.250000,17.330000
2015-01-07,7.19,35.959999,17.510000
2015-01-08,7.24,37.180000,17.559999
2015-01-09,7.15,36.840000,17.350000
2015-01-12,7.29,36.990002,17.340000
2015-01-13,7.58,36.810001,17.389999
2015-01-14,7.49,37.340000,17.200001


In [77]:
# calculate daily change and drop the resulting first null row
stocks_daily_change = stocks.pct_change().dropna()


In [78]:
# create several smaller dataframes to contain the different groups of stocks picked
index_daily_change = stocks_daily_change[index_stocks]
picked_daily_change = stocks_daily_change[picked_stocks]
random_daily_change = stocks_daily_change[random_stocks]


In [107]:
# Create a variance, covariance and beta function

def var_cov_beta(stock_df, index_df):
    """Calculate the beta of a given stock against a given index"""

    variance = stock_df.var()
    covariance = stock_df.cov(index_df)
    beta = covariance / variance

    return print(f"{stock_df.name} has a beta of {beta} when measured against the {index_df.name} index.")


# Create a rolling beta dataframe
def var_cov_beta_rolling(stock_df, index_df, window):
    """Requires a window parameter (integer in days) and Calculate a rolling beta for a given stock and a given index."""

    rolling_variance = stock_df.rolling(window=window).var()
    rolling_covariance = stock_df.rolling(window=window).cov(index_df)

    rolling_beta = rolling_covariance / rolling_variance

    # Plot rolling beta in hvplot
    return rolling_beta.hvplot(figsize=(20, 10), title=f'Rolling {window}-Day Beta of {stock_df.name} and index {index_df.name}')


In [108]:
var_cov_beta(stocks_daily_change['GOOG'], stocks_daily_change['SPY'])

GOOG has a beta of 0.3749968687284355 when measured against the SPY index.


In [98]:
var_cov_beta_rolling(stocks_daily_change['GOOG'], stocks_daily_change['SPY'], 30)

:Curve   [Date]   (0)

In [87]:
# create covariance for each picked stock and one chosen index fund with a nested forLoop

# for x in range(0, len(combined_stocks)):

#     cov_var = combined_stocks[x]

#     variance = stocks_daily_change[cov_var].var()

#     print(f"\nThe variance of {cov_var} is: {variance}")

#     cov_var2 = index_stocks[0]

#     covariance = stocks_daily_change[cov_var].cov(index_daily_change[cov_var2])

#     print(f"The covariance between {cov_var} and {cov_var2} is: {covariance}")

#     print(f"The Beta of {cov_var} and {cov_var2} is {covariance / variance}")
    

In [121]:
# Use Numpy's built-in polynomial-fit feature to find the alpha and beta of a stock and an index

index_s = stocks_daily_change['SPY']
stock_s = stocks_daily_change['GOOG']

beta, alpha = np.polynomial.polynomial.Polynomial.fit(index_s, index_s, 1)
print('Beta for {} stock is = {} and alpha is = {}'.format(stock_s.name, beta, alpha))

Beta for GOOG stock is = 0.004208997626672816 and alpha is = 0.04631586591740075


In [116]:
# Compare iwth Numpy's older 'polyfit' method

beta, alpha = np.polyfit(stock_s, index_s, 1)
print('Beta for {} stock is = {} and alpha is = {}'.format(stock_s.name, beta, alpha))

Beta for GOOG stock is = 0.37499686872843535 and alpha is = 7.120639595721208e-05


In [ ]:
# Further research required for np.polyfit and polynomial.fit methods and their varied results

In [88]:
# calculate risk free rate
# enter following variables as percentages
def risk_free_rate():
    ten_yr_gov_bond = 0.0325
    inflation_rate = .009
    market_return = .06
    beta = var_cov_beta(picked_daily_change['T'], index_daily_change['AGG'])

    risk_free_rate = round((((1 + ten_yr_gov_bond)/(1 + inflation_rate))-1) * 100, 2)

    return print(f"Your risk free rate is: {risk_free_rate}%")

risk_free_rate()

Your risk free rate is: 2.33%
